# Dont overfit ResNet implementation

## Competition Details
#### Link: https://www.kaggle.com/c/dont-overfit-ii
#### Best ResNet Result: 0.517 accuracy
#### Epoch: 10
#### Batch Size: 32
#### Depth: 20

## Import Dependencies

In [5]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
import csv

## Import training and test Datasets

In [6]:
f = open('dataset/train.csv', 'r')
reader = csv.reader(f)


train_array = []
train_label = []

ctr = 0
for row in reader:
    if(ctr):
        train_label.append(int(float(row[1])))
        row.pop(0)
        row.pop(0)
        # print(len(row))
        train_array.append([float(i) for i in row])
        # break
    ctr += 1
f.close()

x_train = np.asarray(train_array)
y_train = np.asarray(train_label)

x_cnn_train = x_train.reshape(250,20,15,1)
y_cnn_train = y_train


print(x_cnn_train.shape)



f = open('dataset/test.csv', 'r')
reader = csv.reader(f)


test_array = []
test_label = []

ctr = 0
for row in reader:
    if(ctr):
        test_label.append(int(float(row[0])))
        row.pop(0)
        test_array.append([float(i) for i in row])
    ctr += 1
f.close()

x_test = np.asarray(test_array)
y_test = np.asarray(test_label)

x_cnn_test = x_test.reshape(19750,20,15,1)
y_cnn_test = y_test

print(x_cnn_test.shape)

(250, 20, 15, 1)
(19750, 20, 15, 1)


## Define Parameters

In [7]:
input_shape = x_cnn_train.shape[1:]

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 10
data_augmentation = True
num_classes = 1
subtract_pixel_mean = True
n = 3
version = 1
depth = 20

## Define Model

In [8]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=4)(x)
    y = Flatten()(x)
    outputs = Dense(1,
                    activation='sigmoid',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



model = resnet_v1(input_shape=input_shape, depth=depth)


model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy', 'binary_crossentropy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 15, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 15, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 20, 15, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 15, 16)   0           batch_normalization_1[0][0]      
_____________________________________

## Train Model

In [9]:
model.fit(x_cnn_train, y_cnn_train, epochs=epochs, batch_size=batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
250/250 [==============================] - 6s 25ms/step - loss: 1.1126 - acc: 0.5600 - binary_crossentropy: 0.9556
Epoch 2/10
250/250 [==============================] - 0s 812us/step - loss: 0.5221 - acc: 0.9000 - binary_crossentropy: 0.3649
Epoch 3/10
250/250 [==============================] - 0s 785us/step - loss: 0.2837 - acc: 0.9960 - binary_crossentropy: 0.1264
Epoch 4/10
250/250 [==============================] - 0s 807us/step - loss: 0.1754 - acc: 1.0000 - binary_crossentropy: 0.0181
Epoch 5/10
250/250 [==============================] - 0s 929us/step - loss: 0.1646 - acc: 1.0000 - binary_crossentropy: 0.0074
Epoch 6/10
250/250 [==============================] - 0s 809us/step - loss: 0.1585 - acc: 1.0000 - binary_crossentropy: 0.0015
Epoch 7/10
250/250 [==============================] - 0s 793us/step - loss: 0.1583 - acc: 1.0000 - binary_crossentropy: 0.0016
Epoch 8/10
250/250 [==============================] - 0s 797us/s

## Run model on Test Data and generate CSV file for submission

In [10]:
filename='RESNET_D'+str(depth)+'_BS'+str(batch_size)+'_E'+str(epochs)+'.csv'

res = model.predict(x_cnn_test)

with open('submissions/'+filename, "w",  newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(('id', 'target'))
        for (x, y) in zip(y_test, res):
            if y[0] >= 0.5:
                targ = 1
            else:
                targ = 0
            writer.writerow((x,targ))
            
print('Done')

Done
